In [ ]:
# Setting up
!pip install transformers
!pip install transformers[sentencepiece]
!pip install datasets

     |████████████████████████████████| 2.8 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.5 MB/s 
     |████████████████████████████████| 895 kB 49.6 MB/s 
     |████████████████████████████████| 636 kB 49.6 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 4.2 MB/s 
     |████████████████████████████████| 270 kB 3.7 MB/s 
     |████████████████████████████████| 1.3 MB 47.8 MB/s 
     |████████████████████████████████| 123 kB 53.4 MB/s 
     |████████████████████████████████| 243 kB 50.4 MB/s 
     |████████████████████████████████| 142 kB 50.0 MB/s 
     |████████████████████████████████| 294 kB 52.3 MB/s 


# A full training
Trainer class를 사용하지 않고 이전 section과 같은 결과를 얻어보자.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Prepare for training
training loop를 짜기 전에, 두 개의 object를 정의해야 함.
- batch들을 반복시키는 dataloader
- tokenized_datasets에 postprocessing을 조금 해줘야 함. (Trainer class는 자동으로 해줬음)

먼저 postprocessing을 해주자. (tokenized_datasets은 각 step에 맞는 method를 갖고 있음)
- 모델이 필요로 하지 않는 columns 제거 (e.g. sentence1, sentence2 columns)
- label column을 labels로 이름을 바꿔야 함
- dataset의 format을 list가 아니라 PyTorch tensor로 지정해줘야 한다.

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence1", "sentence2", "idx"]
) # 필요 없는 column 제거
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") # column 이름 변경
tokenized_datasets.set_format("torch") # format 변경
tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

Dataloader를 정의해보자.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
# data processing이 잘 되었는지 하나의 batch로 확인해보기
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 70]),
 'input_ids': torch.Size([8, 70]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 70])}

data preprocessing 끝! 이제 모델로 가보자.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

# 모든 huggingface transformers 모델들은 labels가 주어졌다면 loss를 return함. loss 뿐만 아니라 logits도 얻을 수 있음
# logits: two for each input in our batch, so a tensor of size 8 x 2.

tensor(0.6033, grad_fn=<NllLossBackward>) torch.Size([8, 2])


training loop를 구성하기 위해 optimizer와 learning rate scheduler를 정의. (Trainer를 구현하기 위해 Trainer default와 똑같은 것 정의)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5) # Trainer default

In [ ]:
from transformers import get_scheduler

num_epochs = 3 # Trainer default
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
print(num_training_steps)

1377


## The training loop

In [ ]:
# 먼저 사용할 device(GPU/CPU)를 정의: GPU 권장

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

## The evaluation loop
huggingface datasets 라이브러리에서 제공하는 metric을 사용해보자.
- metric의 add_batch method를 사용하여 prediction loop (evaluation loop)을 거칠 동안 batch들을 축적하고, 모든 batch들을 축적하면 metric.compute()를 이용해 최종 결과를 얻는다.

In [ ]:
from datasets import load_metric

metric= load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

{'accuracy': 0.8504901960784313, 'f1': 0.8942807625649914}

### Supercharge your training loop with huggingface Accelerate (optional)
multiple GPUs/TPUs를 사용해 distributed training을 하기 위해 HuggingFace Accelerate 라이브러리를 사용할 수 있다.

In [ ]:
from accelerate import Accelerator # 기존 training loop에서 추가된 부분. Accelerator를 import
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator() # 기존 training loop에서 추가된 부분. 환경을 살펴 보고 적절한 분산 설정을 초기화 하는 Accelerator 객체를 인스턴스화

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# 기존 training loop에서 제거된 부분. HuggingFace Accelerate가 device를 배치해주기 때문에 필요 없는 코드임.
''' # 원한다면 device 대신 accelerator.device를 사용할 수 있음.
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
'''

# 기존 training loop에서 추가된 부분. dl은 dataloader를 의미한다.
# accelerator에 dataloader와 model, optimizer를 넣어주면 의도된 분산 training works에 맞게 해당 객체들을 적절한 container에 wrap 해준다. 
train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        # 기존 training loop에서 제거된 부분. device에 직접 보내줄 필요 X
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        
        # loss.backward() 대신 사용
        accelerator.backward(loss)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

분산 환경을 시도해보려면 아래 코드를 실행하자.
- 예시는 https://github.com/huggingface/accelerate/tree/main/examples 참조

In [ ]:
accelerate config

In [ ]:
accelerate launch train.py

In [ ]:
# colab에서 테스트하려면 이걸 실행

from accelerate import notebook_launcher

notebook_launcher(training_function)